In [51]:
import beneath
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import chart_studio.plotly as py
from datetime import datetime, timedelta

from config import BLACKLIST, MODERATORS
from layout import make_layout

In [2]:
# August report
MONTH_NUMBER=8

Find number of stocks w/ at least 3 mentions in a day

In [ ]:
debates_per_day = await beneath.query_warehouse("""
with 
    daily_mentions as (
        select timestamp_trunc(timestamp, day) as date, symbol, count(*) as num_mentions
        from `examples/wallstreetbets_analytics/r_wallstreetbets_comments_stock_mentions`
        group by timestamp_trunc(timestamp, day), symbol
    ),
    daily_totals as (
        select date, count(*) as count
        from daily_mentions
        where num_mentions >= 3
        group by date
    )
SELECT percentiles[offset(25)], percentiles[offset(50)], percentiles[offset(75)]
FROM (SELECT APPROX_QUANTILES(count, 100) percentiles FROM daily_totals)
""")

# Stock mentions

In [3]:
stocks = await beneath.query_warehouse("""
with
    stock_mentions_posts as (
        select 
            symbol, 
            timestamp_trunc(timestamp, day) as day, 
            count(*) as num_mentions
        from `examples/wallstreetbets-analytics/r-wallstreetbets-posts-stock-mentions`
        group by symbol, timestamp_trunc(timestamp, day)
    ),
    stock_mentions_comments as (
        select 
            symbol, 
            timestamp_trunc(timestamp, day) as day, 
            count(*) as num_mentions
        from `examples/wallstreetbets-analytics/r-wallstreetbets-comments-stock-mentions`
        group by symbol, timestamp_trunc(timestamp, day)
    ),
    mentions as (
        select 
            coalesce(p.symbol, c.symbol) as symbol,
            coalesce(p.day, c.day) as day,
            ifnull(p.num_mentions, 0) + ifnull(c.num_mentions,0) as num_mentions
        from stock_mentions_posts p
        full join stock_mentions_comments c on p.symbol = c.symbol and p.day = c.day
    )
select m.symbol, m.day, m.num_mentions, p.close
from mentions m
left join `examples/wallstreetbets-analytics/stock_prices_daily` p on m.symbol = p.symbol and m.day = p.date
order by symbol, day
""")

A bit of feature engineering

In [4]:
# filter out blacklist
stocks = stocks[~stocks['symbol'].isin(BLACKLIST)]

# get the fraction of all mentions in the day (important to do this after applying the blacklist)
mentions_daily = stocks.groupby('day')['num_mentions'].sum().reset_index()
stocks = stocks \
    .merge(mentions_daily, on="day", suffixes=('_stock', '_total')) \
    .sort_values(['symbol','day'])
stocks['fraction_of_mentions'] = stocks['num_mentions_stock'] / stocks['num_mentions_total']
stocks['fraction_of_mentions_MA'] = stocks.groupby('symbol')['fraction_of_mentions'] \
                                    .transform(lambda x: x.rolling(window=7, min_periods=1).mean())

# get the date of peak popularity
symbol_peaks = stocks.loc[stocks.groupby(['symbol'])["fraction_of_mentions_MA"].idxmax()][['symbol', 'day']] \
    .rename(columns={'day': 'date_of_peak_popularity'})
stocks = stocks.merge(symbol_peaks, on='symbol')
stocks['days_from_peak'] = (stocks['day'] - stocks['date_of_peak_popularity']).dt.days

# keep track of the top symbols
top_symbols_alltime_df = stocks \
    .groupby('symbol')['num_mentions_stock'] \
    .sum() \
    .reset_index() \
    .sort_values('num_mentions_stock', ascending=False)
top_symbols_alltime = top_symbols_alltime_df['symbol'][:500]
top_symbols_month_df = stocks.loc[stocks['day'].dt.month == MONTH_NUMBER] \
    .groupby('symbol')['num_mentions_stock'] \
    .sum() \
    .reset_index() \
    .sort_values('num_mentions_stock', ascending=False)
top_symbols_month = top_symbols_month_df['symbol'][:100]

In [5]:
# save top symbols so they're searchable in my web dashboard
await beneath.write_full(
    table_path="examples/wallstreetbets-analytics/top_symbols_alltime",
    records=top_symbols_alltime_df.rename(columns={'num_mentions_stock': 'num_mentions'})[:500],
    key="symbol",
    description="Top 500 symbols of alltime (as of August 30th, 2021)",
    recreate_on_schema_change=True,
)

INFO:beneath:Flushed 500 records to 1 instances (500 total during session)


In [6]:
# assign colors to symbols used in blogpost, for consistency across graphs
symbols_in_blogpost = top_symbols_month[0:10].tolist()
colors = px.colors.qualitative.Plotly
blogpost_colors = dict(zip(symbols_in_blogpost, colors))

## Top symbols of August

In [71]:
fig = px.bar(
    top_symbols_month_df[0:10],
    x="num_mentions_stock",
    y="symbol",
    text="num_mentions_stock",
    color='symbol',
    color_discrete_map=blogpost_colors,
    orientation='h',
    labels={'num_mentions_stock': 'Mentions', 'symbol': 'Symbol'},
)
fig.update_layout(
    make_layout(
        title="Top 10 Most Discussed Symbols in August",
        subtitle="Number of mentions in posts and comments",
        legend=False,
        source_hidden=False,
        override={
            "bargap": 0,
            'showlegend': False,
            'yaxis_title': '',
            'xaxis_zeroline': False
        },
    )
)
fig.update_traces({
    'marker_line_width': 0                 
})
fig.show()

In [72]:
py.plot(fig, filename="wallstreetbets_202108_top10")

'https://plotly.com/~ericpgreen/6/'

## Top meme stocks of alltime

Find stocks that have been over 24% share of mentions

In [8]:
stocks.loc[stocks['fraction_of_mentions'] > .24]['symbol'].unique()

array(['AMC', 'BB', 'GME', 'HOOD', 'MVIS', 'SPCE'], dtype=object)

In [73]:
symbols_to_graph = stocks.loc[stocks['fraction_of_mentions'] > .24]['symbol'].unique()
fig = px.line(stocks.loc[stocks['symbol'].isin(symbols_to_graph)].sort_values('day', ascending=False), 
              x="day",
              y="fraction_of_mentions", 
              line_group="symbol",
              color="symbol",
#               color_discrete_map=blogpost_colors,
              labels={'day': 'Day', 'fraction_of_mentions': 'Percent of mentions (daily)', 'symbol': 'Symbol'},
            )
fig.update_layout(
    make_layout(
        title="Ebbs and Flows of Select Meme Stocks",
        subtitle="Share of mentions across all stocks on r/wallstreetbets",
        source_hidden=False,
        legend=True,
        override={
            'xaxis': {
                'title': ''
            },
            'yaxis': {
                'tickformat': ',.0%'
            }
        },
    )
)
fig.show()

In [74]:
py.plot(fig, filename="wallstreetbets_202108_ebbs_flows")

'https://plotly.com/~ericpgreen/8/'

## Interesting discussions

In [10]:
pd.set_option('max_colwidth', 100)

In [24]:
posts = await beneath.query_warehouse("""
with
    posts_scores_deduped as (
        select r.* from (
            select array_agg(t order by t.__timestamp desc limit 1)[offset(0)] r
            from `examples/reddit/r-wallstreetbets-posts-scores` t
            group by t.__key
        )
    ),
    comments_deduped as (
        select r.* from (
            select array_agg(t order by t.__timestamp desc limit 1)[offset(0)] r
            from `examples/reddit/r-wallstreetbets-comments` t
            where timestamp_trunc(created_on, month) = "2021-08-01"
            group by t.__key
        )
    ),
    posts as (
        select 
            p.created_on, 
            p.id, 
            p.author, 
            p.title, 
            p.text, 
            p.flair, 
            p.permalink,
            s.score,
            array_length(split(p.text, " ")) as num_words,
            array_length(regexp_extract_all(p.title, r"\\x{1F680}")) + array_length(regexp_extract_all(text, r"\\x{1F680}")) as num_rockets,
            array_length(regexp_extract_all(p.title, r"\\x{1F48E}")) + array_length(regexp_extract_all(text, r"\\x{1F48E}")) as num_diamonds,
        from `examples/reddit/r-wallstreetbets-posts` p
        left join posts_scores_deduped s on p.id = s.id
        where timestamp_trunc(created_on, month) = "2021-08-01"
    ),
    comments as (
        select 
            created_on, 
            id, 
            post_id, 
            author, 
            text,
            array_length(split(text, " ")) as num_words,
            array_length(regexp_extract_all(c.text, r"\\x{1F680}")) as num_rockets,
            array_length(regexp_extract_all(c.text, r"\\x{1F48E}")) as num_diamonds,
        from comments_deduped c
    )
select 
    p.created_on, 
    p.title, 
    p.text, 
    p.author, 
    p.flair, 
    p.permalink,
    p.num_words,
    p.score,
    count(c.id) as num_comments,
    sum(c.num_words) as sum_comments_words, 
    sum(c.num_words)/count(c.id) as avg_words_per_comment,
    p.num_rockets + sum(c.num_rockets) as num_rockets,
    (p.num_rockets + sum(c.num_rockets))/count(c.id) as avg_rockets_per_comment,
    count(distinct if(c.num_rockets > 0, c.author, null)) as num_authors_commented_with_rocket,
    count(distinct if(c.num_rockets > 0, c.author, null))/count(distinct c.author) as fraction_of_authors_who_commented_with_rocket,
    p.num_diamonds + sum(c.num_diamonds) as num_diamonds,
    count(distinct c.author) as nunique_commenters,
from posts p
join comments c on p.id = c.post_id
group by p.created_on, p.title, p.text, p.author, p.flair, p.permalink, p.num_words, p.score, p.num_rockets, p.num_diamonds
""")

Exclude moderators

In [25]:
posts_no_mods = posts.loc[[author not in MODERATORS for author in posts['author']]]

Post with the highest avg words per comment (at least 15 comments)

In [26]:
tbl1_df = posts_no_mods[posts_no_mods['num_comments'] >= 15].sort_values('avg_words_per_comment', ascending=False).head(10)

In [76]:
TITLE_LENGTH = 100
table_df = tbl1_df

def format_title(row):
    title = row['title']
    # truncate
    if len(title) > TITLE_LENGTH:
        title = title[0:TITLE_LENGTH] + '...'
    
    # add link
    title = f'<a href="https://reddit.com{row.permalink}">{title}</a>'
    
    return title

fig = go.Figure(data=[go.Table(
    columnwidth = [130,40,25,25,25],
    header=dict(
        values=['<b>Post</b>', '<b>Date</b>', '<b>Score</b>', '<b># of comments</b>', '<b>Average words per comment</b>'],
        fill_color='#1b2442',
        align='left'
    ),
    cells=dict(
        values=[
            table_df.apply(format_title, axis=1),
            table_df.created_on.dt.strftime('%B %d, %Y'),
            table_df.score,
            table_df.num_comments,
            round(table_df.avg_words_per_comment, 1)
        ],
       fill_color='#1b2442',
       align='left',
    )
)])

fig.update_layout(
    make_layout(
        title="Sophisticated intellectual discussion",
        subtitle="Posts with highest average words per comment",
        source_hidden=False,
        override={
            'height': 480
        }
    )
)

fig.show()

In [77]:
py.plot(fig, filename="wallstreetbets_202108_sophisticated")

'https://plotly.com/~ericpgreen/12/'

Post with the highest fraction of comment authors who posted a rocket

This normalizes:
- big threads that of course have lots of rockets
- some small threads where crazy people post a ton of rockets repeatedly

In [28]:
tbl2_df = posts_no_mods[posts_no_mods['num_comments'] >= 25].sort_values('fraction_of_authors_who_commented_with_rocket', ascending=False)[0:10]

In [78]:
TITLE_LENGTH = 100
table_df = tbl2_df

def format_title(row):
    title = row['title']
    # truncate
    if len(title) > TITLE_LENGTH:
        title = title[0:TITLE_LENGTH] + '...'
    
    # add link
    title = f'<a href="https://reddit.com{row.permalink}">{title}</a>'
    
    return title

fig = go.Figure(data=[go.Table(
    columnwidth = [130,40,25,25,25],
    header=dict(
        values=['<b>Post</b>', '<b>Date</b>', '<b>Score</b>', '<b># of comments</b>', '<b>% of commenters who included a 🚀</b>'],
        fill_color='#1b2442',
        align='left'
    ),
    cells=dict(
        values=[
            table_df.apply(format_title, axis=1),
            table_df.created_on.dt.strftime('%B %d, %Y'),
            table_df.score,
            table_df.num_comments,
            table_df.fraction_of_authors_who_commented_with_rocket
        ],
       format = [None, None, None, None, ",.1%"],
       fill_color='#1b2442',
       align='left',
    )
)])

fig.update_layout(
    make_layout(
        title="Stocks approaching liftoff",
        subtitle="Posts with highest percent of commenters who included a 🚀",
        source_hidden=False,
        override={
            'height': 580
        }
    )
)

fig.show()

In [79]:
py.plot(fig, filename="wallstreetbets_202108_rockets")

'https://plotly.com/~ericpgreen/4/'

# Influencers

In [43]:
influencers = await beneath.query_warehouse("""
WITH
    posts_deduped AS (
      SELECT r.* FROM (
        SELECT array_agg(t ORDER BY t.__timestamp DESC LIMIT 1)[OFFSET(0)] r
        FROM `examples/reddit/r-wallstreetbets-posts` t
        GROUP BY t.__key
      )
    ),
    scores_deduped AS (
      SELECT r.* FROM (
        SELECT array_agg(t ORDER BY t.__timestamp DESC LIMIT 1)[OFFSET(0)] r
        FROM `examples/reddit/r-wallstreetbets-posts-scores` t
        GROUP BY t.__key
      )
    )
select 
    author, 
    count(*) as num_posts, 
    sum(score) as total_upvotes, 
    sum(score)/count(*) as avg_upvotes, 
    avg(upvote_ratio) as avg_upvote_ratio, 
    min(created_on) as first_active_date, 
    max(created_on) as last_active_date,
    TIMESTAMP_DIFF(max(created_on), min(created_on), day) as days_active
from posts_deduped p
left join scores_deduped s on p.id = s.id
where flair not in ("Meme", "Gain", "Loss")
group by author
order by total_upvotes desc
""")

In [44]:
influencers = influencers.loc[~influencers['total_upvotes'].isnull()]
influencers['total_upvotes'] = influencers['total_upvotes'].astype('int')

In [45]:
cond1 = [author not in MODERATORS for author in influencers['author']]
cond2 = influencers['last_active_date'] > '2021-07-01'
cond3 = influencers['num_posts'] >= 10
cond4 = influencers['avg_upvote_ratio'] >= .9

In [80]:
fig = px.bar(
    influencers.loc[cond1 & cond2 & cond3][:10],
    x="total_upvotes",
    y="author",
    text="total_upvotes",
    color='author',
    orientation='h',
    labels={'total_upvotes': '# of Upvotes', 'author': 'Author'},
)
fig.update_layout(
    make_layout(
        title="Most Influential Active Authors",
        subtitle="Number of upvotes since March 9th (excluding Memes, Gains, and Losses)",
        legend=False,
        source_hidden=False,
        override={
            "bargap": 0,
            'showlegend': False,
            'yaxis_title': '',
            'xaxis_zeroline': False
        },
    )
)
fig.update_traces({
    'marker_line_width': 0                 
})
fig.show()

In [81]:
py.plot(fig, filename="wallstreetbets_202108_influencers")

'https://plotly.com/~ericpgreen/1/'